## Import 

In [0]:
import math

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [0]:
# check tensorflow version
tf.__version__

In [0]:
# NUM_CLASSES = 10

## Download Kaggle data

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9,ta;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-datasets/663/1265/handwrittenmathsymbols.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1564136253&Signature=RYRQ1iAdRSJ5krXN8HiO9j0BifQqrUxoICFKeSE9V5k406G0UvB4fdbooPRAtlfPfqUvZE49lyvNgF5Vr5cK3idHUAQqppgRdMmnGkJXgENIeK%2B9MtTQslFXpgkVB%2FNkHY%2BBH0SKMgNPOHaoiuD8iDz1pyHLvCe6wsVsCVbPsUBUwHZItr%2FM%2Bh1bpSF0iG8WzCzug%2BWspm7%2FnDNAc2zPUKie0nlRLZUqO5GwG%2Bg2LIok%2BFq8WQCOBHWNE1BS%2FSy3KfLP6noqt5OvjT8LvZyWHf36uRwvPkyTxqifPlbq5cwicPl9utdVnNSEwItCKrvZoctZvYO5SgxZh7kbdoeevQ%3D%3D" -O "handwrittenmathsymbols.zip" -c

In [0]:
# !mkdir data
# !mv ./handwrittenmathsymbols.zip ./data
# !mkdir ./data/mathsymbols
# !unzip ./data/handwrittenmathsymbols.zip -d ./data/mathsymbols
# !unrar x ./data/mathsymbols/data.rar ./data/mathsymbols/

In [0]:
# !rm -rf ./data/mathsymbols/extracted_images/

## Load data

In [0]:
train_data_dir = './data/mathsymbols/extracted_images/'
batch_size = 64
img_height, img_width = 45,45
nb_epochs = 5

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
#     class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
#     class_mode='binary',
    subset='validation') # set as validation data

In [0]:
NUM_CLASSES = 82

## Build & Train model

In [0]:
## Model 
inputs = Input(shape=(45, 45, 3), name='input')

x = Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)

x = Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)


x = Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)

x = tf.keras.layers.Flatten()(x)
x = Dense(200)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

predications = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [0]:
lr_decay = lambda epoch: 0.001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

In [0]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, 
#                     validation_data=(x_test, y_test), callbacks=[decay_callback])

# model.load_weights('mathsymbol.h5')

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 5, callbacks=[decay_callback])

In [0]:
# Save & Export

model.save('mathsymbol-best.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('mathsymbol-best.h5')
tflite_model = converter.convert()
open('mathsymbol-best.tflite', 'wb').write(tflite_model)

## Download tflite


In [0]:
try:
    from google.colab import files
    files.download('mathsymbol-best.tflite')
except:
    print("Skip downloading")